In [3]:
import pandas as pd
from pathlib import Path
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
PROJECT_DIR = Path().cwd().parent
DATA_DIR = PROJECT_DIR.joinpath('data')

In [ ]:
import argparse
import json
import os
import threading
import time
from random import random
import numpy as np
import requests
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers, initializers, losses, metrics, regularizers
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
import yaml
from sklearn.model_selection import train_test_split
import pandas as pd
import sys
import pickle
from keras.utils import to_categorical

# tf.random.set_random_seed(42)  # tensorflow seed fixing
# %%
'''
    - 데이터 로딩하기
'''

with open('/home/wonseok/PPFL/4_ppfl_simulation/feature_book.yaml') as f:
    feature_book = yaml.load(f, Loader=yaml.FullLoader)
   
   
with open('/home/wonseok/PPFL/4_ppfl_simulation/hicu/dataset.pkl','rb') as f:
    dataset = pickle.load(f)

# #%%
# data = pd.read_csv('/home/wonseok/PPFL/4_ppfl_simulation/hicu/data/HICU.csv')
# feature = data.drop(columns = ['death','연구등록번호'])
# feature_all = data[feature_book["common_features"] + feature_book['hicu_specific']]
# label = data['death']

# train_X, test_X, train_Y, test_Y = train_test_split(feature_all, label, test_size=0.2, stratify=label, random_state=0)
# train_X, valid_X, train_Y, valid_Y = train_test_split(train_X, train_Y, test_size=0.25, stratify=train_Y, random_state=0)

# # common만 선정
# train_X, valid_X, test_X = train_X[feature_book['common_features']], valid_X[feature_book['common_features']], test_X[feature_book['common_features']]

2022-07-11 09:12:53.418967: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-11 09:12:53.425479: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/rh/devtoolset-9/root/usr/lib64:/opt/rh/devtoolset-9/root/usr/lib:/opt/rh/devtoolset-9/root/usr/lib64/dyninst:/opt/rh/devtoolset-9/root/usr/lib/dyninst:/opt/rh/devtoolset-9/root/usr/lib64:/opt/rh/devtoolset-9/root/usr/lib:/usr/local/cuda/lib64::/opt/rh/devtoolset-9/root/usr/lib64:/opt/rh/devtoolset-9/root/usr/lib:/opt/rh/devtoolset-9/root/usr/lib64/dyninst:/opt/rh/devtoolset-9/root/usr/lib/dyninst:/opt/rh/devtoolset-9/root/usr/lib64:/opt/rh/devtoolse

In [4]:
len(feature_book['hicu_specific'])

13

In [5]:
dataset['train_X'][feature_book['common_features'] + feature_book['hicu_specific']]

,height,heartrate,weight,creatinine,bun,temperature,hct,bp,age,gender,...,Cl,Eosinophil(%),체중P,체중Z,신장P,신장Z,Hemoglobin,체표면적,PLT Count,Basophil(%)
4849,0.375276,0.278302,0.229508,0.000859,0.079399,0.500000,0.634518,0.394472,0.400000,1.0,...,0.536313,0.019505,0.66800,0.914597,0.276000,0.647788,0.644366,0.426934,0.271087,0.117647
2786,0.373510,0.287736,0.237049,0.001810,0.073319,0.607143,0.628596,0.572864,0.423529,1.0,...,0.486034,0.008502,0.74200,0.917909,0.230000,0.645934,0.690141,0.432665,0.265742,0.058824
3006,0.363218,0.216981,0.225246,0.001002,0.123033,0.517857,0.653553,0.447236,0.317647,1.0,...,0.525140,0.012003,0.62100,0.912646,0.277589,0.642549,0.679578,0.404735,0.186905,0.147059
2869,0.316556,0.231132,0.156066,0.000736,0.093705,0.553571,0.550761,0.409548,0.717647,0.0,...,0.525140,0.019505,0.12500,0.890050,0.000000,0.607666,0.584507,0.300860,0.244029,0.147059
3291,0.348786,0.245283,0.084262,0.001561,0.177873,0.428571,0.523971,0.733668,0.517647,0.0,...,0.484171,0.009502,0.00000,0.743821,0.293000,0.648407,0.584507,0.209169,0.123100,0.029412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,0.381236,0.198113,0.217377,0.000757,0.100858,0.535714,0.591371,0.472362,0.564706,1.0,...,0.581006,0.006002,0.52500,0.908824,0.455000,0.653997,0.612676,0.415473,0.168866,0.029412
3553,0.375276,0.202830,0.222951,0.001554,0.107296,0.500000,0.522843,0.555276,0.717647,1.0,...,0.502793,0.111028,0.59400,0.911563,0.276000,0.647788,0.549296,0.418338,0.107733,0.117647
4220,0.363218,0.231132,0.219408,0.001125,0.045541,0.482143,0.723350,0.479899,0.058824,1.0,...,0.532588,0.012753,0.60239,0.912181,0.277589,0.642549,0.802817,0.404735,0.148321,0.102941
2635,0.350993,0.250000,0.200000,0.000532,0.063662,0.660714,0.487310,0.630653,0.376471,0.0,...,0.502793,0.012003,0.81900,0.921948,0.365000,0.651009,0.570423,0.375358,0.312176,0.147059


In [6]:
dataset['train_X'][feature_book['common_features'] + feature_book['hicu_specific']]

KeyError: "['Monocyte(%)', 'Neutrophil(%)', '외국인여부', 'Cl', 'Eosinophil(%)', '체중P', '체중Z', '신장P', '신장Z', 'Hemoglobin', '체표면적', 'PLT Count', 'Basophil(%)'] not in index"

In [5]:
def build_nn_model(
        input_size=len(feature_book['common_features']), n_layers=3, n_hidden_units=30,
        random_seed=None, num_classes=2
):
    """
        creates the MLP network
        :return: model: models.Model`
        """
    # create input layer
    input_layer = layers.Input(shape=input_size, name="input")
    # create intermediate layer
    dense = input_layer
    for i in range(n_layers):
        dense = layers.Dense(
            units=n_hidden_units,
            kernel_initializer=initializers.glorot_uniform(seed=random_seed),
            bias_initializer='zeros',
            activation='relu',
            name='intermediate_dense_{}'.format(i + 1)
        )(dense)
    output_layer = layers.Dense(num_classes,
                                kernel_initializer=initializers.glorot_uniform(seed=random_seed),
                                bias_initializer='zeros',
                                activation='softmax',
                                name='classifier')(dense)
    model = models.Model(input_layer, output_layer)
    model.compile(optimizer=tf.keras.optimizers.SGD(),
                         loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                         metrics=['accuracy'])
    return model

In [ ]:
model = build_nn_model()

In [13]:
with open('/home/wonseok/PPFL/4_ppfl_simulation/hicu/final_global_weight.pkl','rb') as f:
    weights = pickle.load(f)

In [28]:
model.set_weights(weights)